In [1]:
import keras
from keras.models import Model
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.layers import *
import pandas as pd
import numpy as np
from position import Position
import matplotlib.pyplot as plt
%matplotlib inline
import imp
# imp.reload(cnnfeat)
from multiprocessing import Queue
from multiprocessing import Manager, Process, Pool
import os
from time import time
import pickle
import bcolz
from tqdm import tqdm 

Using TensorFlow backend.


Initializing Magics


In [2]:
def bitfield(bitboard):
    return np.array([1 if b == '1' else 0 for b in bin(bitboard)[2:].zfill(64)]).reshape((8,8))

In [3]:
def get_feats(pos):
    krights = 0
    if preserved_kingside_castle_rights(pos.position_flags, Side.W):
        krights |= G1 | F1
    if preserved_queenside_castle_rights(pos.position_flags, Side.W):
        krights |= C1 | D1
    if preserved_kingside_castle_rights(pos.position_flags, Side.B):
        krights |= G8 | F8
    if preserved_kingside_castle_rights(pos.position_flags, Side.B):
        krights |= C8 | D8
    krights_plane = bitfield(krights)
    stm_plane = np.zeros(64).reshape((8,8)) if pos.side_to_move() == S.W else np.ones(64).reshape((8,8))
    wp_plane = bitfield(pos.pieces[Pt.piece(Pt.P, S.W)])
    bp_plane = bitfield(pos.pieces[Pt.piece(Pt.P, S.B)])
    wn_plane = bitfield(pos.pieces[Pt.piece(Pt.N, S.W)])
    bn_plane = bitfield(pos.pieces[Pt.piece(Pt.N, S.B)])
    wb_plane = bitfield(pos.pieces[Pt.piece(Pt.B, S.W)])
    bb_plane = bitfield(pos.pieces[Pt.piece(Pt.B, S.B)])
    wr_plane = bitfield(pos.pieces[Pt.piece(Pt.R, S.W)])
    br_plane = bitfield(pos.pieces[Pt.piece(Pt.R, S.B)])
    wq_plane = bitfield(pos.pieces[Pt.piece(Pt.Q, S.W)])
    bq_plane = bitfield(pos.pieces[Pt.piece(Pt.Q, S.B)])
    wk_plane = bitfield(pos.pieces[Pt.piece(Pt.K, S.W)])
    bk_plane = bitfield(pos.pieces[Pt.piece(Pt.K, S.B)])
    return np.stack([krights_plane, stm_plane, wp_plane, bp_plane, wn_plane, bn_plane, wb_plane, bb_plane,
                     wr_plane, br_plane, wq_plane, bq_plane, wk_plane, bk_plane], axis=-1) # 8x8x14

In [5]:
sf_scores3 = pd.read_pickle('/ssd/slonik_data/sf_scores_3dep.pkl')

In [17]:
xi = np.random.randint(0, len(sf_scores3), size=10)
rows = sf_scores3.loc[xi]
fens = rows.fen.tolist()
scores = np.array(rows.stockfish_score, dtype='float32')

In [18]:
for f,s in zip(fens,scores):
    print(Position.from_fen(f))
    print(s, "\n")

W to move
 ·  ·  ·  ♖  ·  ♝  ♞  ♜ 
 ·  ♚  ♟  ·  ·  ·  ♟  · 
 ♟  ♟  ·  ·  ♝  ♟  ·  · 
 ·  ·  ·  ·  ·  ·  ·  ♟ 
 ·  ·  ♟  ·  ♙  ♗  ·  · 
 ·  ·  ♘  ·  ·  ♙  ·  · 
 ♙  ♙  ♙  ·  ·  ·  ♙  ♙ 
 ·  ·  ·  ♖  ·  ·  ♔  · 

0.111094 

W to move
 ·  ·  ·  ·  ·  ·  ·  · 
 ·  ·  ·  ·  ♚  ·  ·  · 
 ·  ·  ♟  ·  ♜  ·  ·  · 
 ·  ·  ♙  ·  ·  ♟  ·  ♟ 
 ·  ·  ·  ♖  ·  ·  ·  · 
 ·  ·  ·  ·  ·  ·  ♙  · 
 ♞  ·  ·  ·  ·  ♙  ♗  ♙ 
 ·  ·  ·  ·  ·  ♔  ·  · 

0.0526805 

W to move
 ♜  ·  ♝  ·  ♚  ♝  ♞  ♜ 
 ♟  ♟  ·  ♟  ♟  ♟  ♟  ♟ 
 ·  ·  ·  ·  ·  ·  ·  · 
 ·  ·  ·  ·  ·  ·  ·  · 
 ·  ·  ·  ♞  ♙  ·  ·  · 
 ·  ·  ·  ·  ·  ♙  ·  · 
 ♙  ♙  ♙  ·  ·  ·  ♙  ♙ 
 ♖  ♘  ♗  ·  ♔  ♗  ·  ♖ 

-0.0218469 

B to move
 ♜  ♞  ♝  ♛  ♚  ·  ♞  ♜ 
 ♟  ♟  ·  ·  ·  ♟  ♟  ♟ 
 ·  ·  ♟  ·  ♟  ·  ·  · 
 ·  ·  ·  ·  ·  ·  ·  · 
 ·  ♝  ♙  ♙  ♘  ·  ·  · 
 ·  ·  ·  ·  ·  ·  ·  · 
 ♙  ♙  ·  ♗  ·  ♙  ♙  ♙ 
 ♖  ·  ·  ♕  ♔  ♗  ♘  ♖ 

0.0361016 

B to move
 ·  ·  ·  ·  ·  ·  ·  ♚ 
 ·  ♟  ·  ·  ·  ·  ·  ♟ 
 ·  ♜  ·  ♟  ·  ·  ·  ♜ 
 ♟  ·  ·  ♗  ·  ♙  ♟  ·

In [48]:
fv = bcolz.open(mode='r', rootdir='/ssd/slonik_data/features_valid.bc')

In [24]:
bcolz.open(mode='r', rootdir='/ssd/slonik_data/features_sf12_origfen_valid.bc')

carray((60000, 8, 8, 14), float64)
  nbytes := 410.16 MB; cbytes := 26.67 MB; ratio: 15.38
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 1024; chunksize: 7340032; blocksize: 32768
  rootdir := '/ssd/slonik_data/features_sf12_origfen_valid.bc'
  mode    := 'r'
[[[[ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  1.]
   [ 0.  1.  0. ...,  0.  0.  0.]]

  [[ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]]

  [[ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   ..., 
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  1.  0. ...,  0.  0.  0.]]

  ..., 
  [[ 0.  1.  1. ...,  0.  0.  0.]
   [ 0.  1.  0. ..

In [45]:
from cnnfeat import get_feats

In [94]:
np.allclose(get_feats(Position.from_fen(positions.iloc[5].fen)), pos_feats[5])

True

In [52]:
positions = pd.read_pickle('/ssd/slonik_data/positions.pkl')

In [53]:
len(positions)

700759

In [10]:
features_sf3_valid = bcolz.open(mode='r', rootdir='/ssd/slonik_data/features_sf3_valid.bc')

In [58]:
len(pos_sf3_feats) - len(positions), len(sf_scores3) - 60000

(235705, 235705)

In [11]:
blen = features_sf3_valid.chunklen
dtype = features_sf3_valid.dtype
nb_items = len(features_sf3_valid)//10
itr = bcolz.iterblocks(features_sf3_valid, blen=blen, start=0, stop=nb_items)
feats = bcolz.carray(np.empty((0,8,8,14)), mode='w', chunklen=1024, rootdir='/ssd/slonik_data/features_sf3_valid_sample.bc')
for i in itr:
    feats.append(i)
feats.flush()

In [14]:
from cnnfeat import *